In [2]:
##
## Notebook Paramaters
##
transient_tables = True

print("Paramaters for this run:")
print(f"transient_tables = {transient_tables}")

StatementMeta(, c2df6915-7f77-4648-a831-0e623f5b6ec9, 4, Finished, Available, Finished)

Paramaters for this run:
transient_tables = True


In [3]:
from pyspark.sql import SparkSession

app_name = "DropLakehouseTables"

# Get the current Spark session
spark = SparkSession.builder \
    .appName(app_name) \
    .getOrCreate()

print(f"Spark session {app_name} has been created successfully.")

StatementMeta(, c2df6915-7f77-4648-a831-0e623f5b6ec9, 5, Finished, Available, Finished)

Spark session DropLakehouseTables has been created successfully.


In [4]:
layer_to_schema = {
    "default": "dbo",
    "bronze": "brz",
    "silver": "slv",
    "gold": "gld"
}

print("The dictionary 'layer_to_schema' has been created successfully.")

StatementMeta(, c2df6915-7f77-4648-a831-0e623f5b6ec9, 6, Finished, Available, Finished)

The dictionary 'layer_to_schema' has been created successfully.


In [ ]:
# Configure layer and schema where tables are to be deleted
layer = "bronze"
db_schema = layer_to_schema.get(layer, "NOT_FOUND")
application = "warehouse"

# Define the OneLake folder path
workspace_id = " [ YOUR ID HERE ] " ## Adv Wrks DE 3 Dev
lakehouse_id = " [ YOUR ID HERE ] " ## AdventureWorks_Lakehouse
folder = "/Files/" + layer + "/" + application
folder_path = "abfss://" + workspace_id + "@onelake.dfs.fabric.microsoft.com/" + lakehouse_id + folder

print(f"Configured to process from '{layer}' layer files in:\n{folder_path}")

StatementMeta(, c2df6915-7f77-4648-a831-0e623f5b6ec9, 7, Finished, Available, Finished)

Configured to process from 'bronze' layer files in:
abfss://3ac7ce42-ae74-4e7d-8ac3-5ce8358a30df@onelake.dfs.fabric.microsoft.com/50402dac-ce50-4831-af2b-7d65ca8fe7db/Files/bronze/warehouse


In [7]:
# Get all files in the folder
file_list = spark.read.format("binaryFile").load(folder_path).select("path").collect()

# Drop all tables to to associated CSV data
for file in sorted(file_list):
    file_path = file["path"]
    if file_path.endswith(".csv"):
        # Extract the table name from the file name
        table_name = file_path.split("/")[-1].split(".")[0]
        
        # Drop the table
        if transient_tables:
            full_table_name = db_schema + "." + layer_to_schema.get("silver", "NOT_FOUND") +"_" + table_name
        else:
            full_table_name = db_schema + "." + table_name
        
        spark.sql(f"DROP TABLE IF EXISTS {full_table_name}")
        print(f"Dropped table {full_table_name}")

StatementMeta(, c2df6915-7f77-4648-a831-0e623f5b6ec9, 9, Finished, Available, Finished)

Dropped table brz.slv_DimAccount
Dropped table brz.slv_DimCurrency
Dropped table brz.slv_DimCustomer
Dropped table brz.slv_DimDate
Dropped table brz.slv_DimDepartmentGroup
Dropped table brz.slv_DimEmployee
Dropped table brz.slv_DimGeography
Dropped table brz.slv_DimOrganization
Dropped table brz.slv_DimProduct
Dropped table brz.slv_DimProductCategory
Dropped table brz.slv_DimProductSubcategory
Dropped table brz.slv_DimPromotion
Dropped table brz.slv_DimReseller
Dropped table brz.slv_DimSalesReason
Dropped table brz.slv_DimSalesTerritory
Dropped table brz.slv_DimScenario
Dropped table brz.slv_FactAdditionalInternationalProductDescription
Dropped table brz.slv_FactCallCenter
Dropped table brz.slv_FactCurrencyRate
Dropped table brz.slv_FactFinance
Dropped table brz.slv_FactInternetSales
Dropped table brz.slv_FactInternetSalesReason
Dropped table brz.slv_FactProductInventory
Dropped table brz.slv_FactResellerSales
Dropped table brz.slv_FactSalesQuota
Dropped table brz.slv_FactSurveyRespons

In [8]:
# Stop the Spark session
# NOTE: frees up limited F2 SKU capacity resources
spark.stop()

print("Spark session has been stopped successfully.")

StatementMeta(, c2df6915-7f77-4648-a831-0e623f5b6ec9, 10, Finished, Available, Finished)

Spark session has been stopped successfully.
